# Controle da carteira de ações (Swing Trade)


- **Não funciona para Day Trade.**

## Baixando a planinlha com os ativos negociados

Para pegar a planinlha de excel com todas as negociações é só ir no site da [B3](http://www.b3.com.br/pt_br/), procurar pelo Canal Eletrônico do Investidor (CEI).

> [Link direto](https://cei.b3.com.br/CEI_Responsivo/login.aspx) 

Depois de acessar a página é só ir em: Extratos e informativos $\Rightarrow$ Negociações de Ativos. Você deve ver a opção de baixar a planilha por excel nesta página.

### Referências para o programa.

[Link](https://gspread.readthedocs.io/en/latest/api.html#module-gspread.utils) de uma documentação
das planilhas do google.

[Link](https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html) descrevendo como fazer para pegar as autenticações do google spreadsheet.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [42]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('cred.json', scope)
client = gspread.authorize(creds)

In [4]:
def cleaning_and_organizing(df):

    df.rename(columns = {'Preço (R$)':'Preço de C/V', 'Data Negócio': 'Data',
                         'Valor Total (R$)':'Valor Total',
                         'Especificação do Ativo':'Especificação',
                         'C/V':'Tipo de Execução',
                         'Quantidade': 'Quantidade de C/V'}, inplace=True);

    df['Mercado'] = df['Mercado'].apply(lambda x: x.replace('Merc.',''))
    df['Quantidade de C/V'] = df['Quantidade de C/V'].astype(np.int16)
    df['Código'] = df['Código'].apply(lambda x: x[:-1])
    df['Tipo de Execução'] = df['Tipo de Execução'].str.strip()
    df['Mercado'] = df['Mercado'].str.strip()
    df['Especificação'] = df['Especificação'].str.strip()
    df['Código'] = df['Código'].str.strip()
    
    df.loc[df['Tipo de Execução'] == 'V', ['Quantidade de C/V']] *= -1
    
    return df

In [5]:
def save_last(df_all):
    
    df_carteira = pd.DataFrame(columns = df_all.columns)
    df_carteira['Data Negócio'] = carteira['Data']
    df_carteira['C/V'] = 'C'
    df_carteira['Quantidade'] = carteira['Quantidade Atual']
    df_carteira['Código'] = carteira['Código'].apply(lambda x: str(x) + 'F')
    df_carteira['Preço (R$)'] = carteira['<Preço>']
    df_carteira['Valor Total (R$)'] = carteira['<Valor Total>']
    df_carteira['Mercado'] = 'Merc. Fracionário'

    for cod in df_carteira['Código']:

        df_carteira.loc[df_carteira['Código'] == cod, 'Especificação do Ativo'] = df_all.loc[df_all['Código'] == cod, 'Especificação do Ativo'].iloc[-1]

    return df_carteira

In [6]:
def calc_price_and_gain(tipo_op, preco_atual_anterior, preco_cv, qnt_atual_anterior, qnt_cv):

    if tipo_op == 'C':

        mean = np.average([preco_atual_anterior, preco_cv], weights = [qnt_atual_anterior, qnt_cv]) 
        return mean, 0.0

    elif tipo_op == 'V':

        mean = preco_atual_anterior
        lucro = (preco_cv - preco_atual_anterior) * np.abs(qnt_cv)
        return mean, lucro

    elif (tipo_op == 'DES') | (tipo_op == 'AGR'):

        mean = preco_atual_anterior * preco_cv
        return mean, 0.0
    
    return np.nan

In [7]:
def stock_mean_price(df_acao):
    
    ''' Calcula o preço médio da ação e em caso de venda o lucro/prejuízo. '''
    
    preco_cv = df_acao['Preço de C/V']

    quantidade_cv = df_acao['Quantidade de C/V']

    quantidade_atual = df_acao['Quantidade Atual']

    tipo_op = df_acao['Tipo de Execução']

    mean_price = np.zeros(df_acao.shape[0], dtype=np.float64)
    gain_loss = np.zeros(df_acao.shape[0], dtype=np.float64)

    mean_price[0] = preco_cv[0]

    for i in range(1, df_acao.shape[0]):

        mean_price[i], gain_loss[i] = calc_price_and_gain(tipo_op[i],
                                                          mean_price[i - 1], preco_cv[i],
                                                          quantidade_atual[i - 1], quantidade_cv[i])
    
    return mean_price[:], gain_loss[:]        

In [8]:
def combine_InfoCEI_files(arq_names):
    
    df = pd.DataFrame()
    
    for name in arq_names:
            
        if name.find('.csv') != -1:
            
            aux = pd.read_csv(name)
        
            aux['Data Negócio'] = pd.to_datetime(aux['Data Negócio'])
        
            df = pd.concat([df, aux], ignore_index=True)            
            
        elif name.find('.xls') != -1:
                
            aux = pd.read_excel(name, header=10, skipfooter=4).dropna(axis=1)
            
            aux['Data Negócio'] = aux['Data Negócio'].apply(lambda x: x.replace('/','-'))
            aux['Data Negócio'] = pd.to_datetime(aux['Data Negócio'], infer_datetime_format=True, dayfirst=True)
        
            df = pd.concat([df, aux], ignore_index=True)
    
    df.sort_values(by='Data Negócio', ascending=True, inplace=True)
    
    return df.reset_index(drop=True)

In [9]:
def get_market_values(names):
    
    market_values = client.open('Google_Finance').sheet1

    size = names.shape[0]
    
    cell_list = market_values.range('A1:A' + str(size))

    for i, cell in enumerate(cell_list):
        cell.value = names[i]
        market_values.update_acell('B' + str(i + 1), '=GOOGLEFINANCE(A' + str(i + 1) + ';"price")')

    market_values.update_cells(cell_list);

    market_values = market_values.col_values(2)[0:size]
    
    market_values = [float(market_values[i].replace(',', '.')) for i in range(size)]
      
    return np.asarray(market_values)

In [10]:
def intersection(l1, l2):
    return list(set(l1) & set(l2))

In [11]:
def carteira_de_acoes_atual(df):
    
    carteira = pd.DataFrame(columns=df.columns)

    cods = df['Código'].unique()
    
    for cod in cods:

        last_idx = df[df[['Código']] == cod].last_valid_index() 

        carteira = carteira.append(df.iloc[last_idx], ignore_index=True)

    carteira = carteira[carteira['Quantidade Atual'] != 0.0]
    
    carteira.sort_values(by='Data', ascending=True, inplace=True)
    
    drop_columns = ['Especificação', 'Tipo de Execução', 'Quantidade de C/V', 'Mercado', 'Preço de C/V']
    
    carteira.drop(drop_columns, inplace=True, axis=1)
    
    carteira.rename(columns={'Preço Atual': '<Preço>'}, inplace=True)
    
    carteira.reset_index(drop=True, inplace=True)
    
    carteira['<Valor Total>'] = carteira['Quantidade Atual'] * carteira['<Preço>']
    
    carteira['Preço Atual'] = get_market_values(carteira['Código'])

    carteira['Valor Total Atual'] = carteira['Quantidade Atual'] * carteira['Preço Atual']
    
    carteira['L/P'] = carteira['Quantidade Atual'] * ( carteira['Preço Atual'] - carteira['<Preço>'] )

    carteira['L/P (%)'] = (carteira['L/P'] / (carteira['<Preço>'] * carteira['Quantidade Atual'])) * 100.0

    carteira['L/P'] = carteira['L/P'].round(2)

    carteira['L/P (%)'] = carteira['L/P (%)'].round(2) 
    
    carteira['<Preço>'] = carteira['<Preço>'].round(2)
    
    
    cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>',
                      'Preço Atual', 'Valor Total Atual', 'L/P', 'L/P (%)']

    return carteira[cols_ordenadas]

In [12]:
def update_carteira_online(carteira):

    carteira['Data'] = carteira['Data'].dt.strftime('%d-%m-%Y')

    carteira['Preço Atual'] = ['=GOOGLEFINANCE(B' + str(i) + '; "price")' for i in range(2, carteira.shape[0]+2)]

    carteira['<Valor Total>'] = ['=D' + str(i) + ' * C' + str(i) for i in range(2, carteira.shape[0]+2)]

    carteira['Valor Total Atual'] = ['=F' + str(i) + ' * C' + str(i) for i in range(2, carteira.shape[0]+2)]

    carteira['L/P'] = ['=G' + str(i) + ' - E' + str(i) for i in range(2, carteira.shape[0]+2)]

    carteira['L/P (%)'] = ['=(H' + str(i) + ' / E' + str(i) + ')' for i in range(2, carteira.shape[0]+2)]

    google_sheet = client.open('Carteira de Ações').sheet1
    google_sheet.clear()

    google_sheet.insert_row(carteira.columns.to_list(), index=1)

    for row in range(carteira.shape[0]):
        google_sheet.insert_row(carteira.iloc[row].to_list(), index=row+2, value_input_option='USER_ENTERED')
        

## Inserir os arquivos da B3 (InfoCEI.xls)

In [20]:
# Lendo um único InfoCEI
#df_all = pd.read_excel('InfoCEI (1).xls', header=10, skipfooter=4).dropna(axis=1)

# Lendo vários arquivos InfoCEI
arq_names = ['InfoCEI.xls', 'InfoCEI (1).xls']#, 'Ultima_carteira_2019.csv']
df_all = combine_InfoCEI_files(arq_names)

# Lendo a última carteira de ativos do ano anterior
#df_ativos = pd.read_csv('Ultima_carteira_2019.csv')


df_ativos = df_all.copy()
df_des_agr = pd.read_excel('desdobramento_agrupamento_2019.xls')

WARNING *** file size (30195) not 512 + multiple of sector size (512)
WARNING *** file size (32477) not 512 + multiple of sector size (512)


In [21]:
df_ativos.head()

,Data Negócio,C/V,Mercado,Código,Especificação do Ativo,Quantidade,Preço (R$),Valor Total (R$)
0,2019-07-11,C,Merc. Fracionário,RAIL3F,RUMO S.A. ON NM,20,22.17,443.40
1,2019-07-11,C,Merc. Fracionário,PETR4F,PETROBRAS PN N2,17,28.17,478.89
2,2019-07-11,C,Merc. Fracionário,CVCB3F,CVC BRASIL ON NM,10,53.00,530.00
3,2019-07-11,C,Merc. Fracionário,VALE3F,VALE ON NM,10,52.00,520.00
4,2019-07-19,V,Merc. Fracionário,CVCB3F,CVC BRASIL ON NM,8,48.70,389.60


In [22]:
# Limpando e arruamndo os dados
df_ativos = cleaning_and_organizing(df_ativos)
df_des_agr['Data'] = pd.to_datetime(df_des_agr['Data'], infer_datetime_format=True, dayfirst=True)

In [23]:
# Tira a média diária das vendas ou compras. Isso é necessário porque podemos comprar
# ações com valores diferentes durante o pregão em um mesmo dia.

group = df_ativos.groupby(['Data', 'Código', 'Mercado', 'Especificação', 'Tipo de Execução'])

df_ativos = group[['Quantidade de C/V', 'Valor Total']].sum().reset_index()

df_ativos['Preço de C/V'] = np.abs(df_ativos['Valor Total'] / df_ativos['Quantidade de C/V'].astype(np.float64))

# Adicionando algumas colunas.
df_ativos['Preço Atual'] = 0.0
df_ativos['L/P'] = 0.0
df_ativos['Quantidade Atual'] = 0.0

# Adiciona os agrupamentos e desdobramentos.


# Print do dataset até agora.
#df_ativos

In [24]:
cods = intersection(df_ativos['Código'].unique(), df_des_agr['Código'].unique())

for cod in cods:

    df = df_des_agr[df_des_agr['Código'] == cod]
    
    for _, (data_split, _, prop1, prop2) in df.iterrows():

        mask = (df_ativos['Código'] == cod) & (df_ativos['Data'] < data_split)

        row = df_ativos[mask].tail(1).copy()
        
        # Taxas para converter os valores que serão desdobrados/agrupados.
        ratio1 = prop1 / float(prop2)
        ratio2 = prop2 / float(prop1)

        if ratio1 > ratio2:
            row['Tipo de Execução'] = 'DES'
        else:
            row['Tipo de Execução'] = 'AGR'
        
        row['Quantidade de C/V'] = df_ativos.loc[mask, 'Quantidade de C/V'].sum() * (ratio1 - 1.0)
        row['Valor Total'] = ratio1
        row['Preço de C/V'] = ratio2
        
        row.reset_index(inplace=True)
        row['index'] += 1
        row.set_index('index', inplace=True)

        mask = (df_ativos['Data'] < data_split)
        
        df_ativos.reset_index(inplace=True)
        df_ativos.loc[-mask, 'index'] += 1
        df_ativos.set_index('index', inplace=True)

        df_ativos = df_ativos.append(row).sort_values(by='index')
        df_ativos.reset_index(drop=True, inplace=True)
        del row

In [25]:
cods = df_ativos['Código'].unique()

for cod in cods:
    
    mask = (df_ativos['Código'] == cod)

    df_ativos.loc[mask, 'Quantidade Atual'] = df_ativos.loc[mask, 'Quantidade de C/V'].cumsum()

    mean_price, gain_loss = stock_mean_price(df_ativos[mask].reset_index(drop=True))

    df_ativos.loc[mask, 'Preço Atual'] = mean_price
    df_ativos.loc[mask, 'L/P'] = gain_loss

#### Salva um arquivo com as movimentações diárias de cada ação

In [26]:
#df_ativos.to_csv('Dataset_ações_2019.csv')

### Carteira Atual

In [43]:
sheet = client.open("Carteira de Ações").sheet1

In [44]:
carteira = carteira_de_acoes_atual(df_ativos)
carteira

,Data,Código,Quantidade Atual,<Preço>,<Valor Total>,Preço Atual,Valor Total Atual,L/P,L/P (%)
0,2019-09-05,PETR4,39.0,26.38,1028.68,22.83,890.37,-138.31,-13.45
1,2019-09-05,VALE3,25.0,49.80,1244.95,44.61,1115.25,-129.70,-10.42
2,2019-09-05,MGLU3,33.0,34.86,1150.27,45.33,1495.89,345.62,30.05
3,2019-09-05,CYRE3,21.0,24.16,507.36,27.75,582.75,75.39,14.86
4,2019-09-12,VVAR3,100.0,7.76,775.80,11.56,1156.00,380.20,49.01
5,2019-10-01,EZTC3,15.0,36.42,546.30,44.10,661.50,115.20,21.09
6,2019-11-04,JBSS3,21.0,28.49,598.39,21.01,441.21,-157.18,-26.27
7,2019-11-04,LCAM3,29.0,17.75,514.75,19.15,555.35,40.60,7.89
8,2019-12-03,RAIL3,21.0,25.22,529.62,19.30,405.30,-124.32,-23.47
9,2019-12-03,KLBN11,29.0,18.15,526.35,20.98,608.42,82.07,15.59


In [45]:
print('Lucro ao zerar a carteira: {0}'.format(carteira['L/P'].sum()))

Lucro ao zerar a carteira: 489.56999999999994


### Salva a última carteira de ativos do ano

In [22]:
#df_carteira = save_last(df_all)
#df_carteira.to_csv('Ultima_carteira_2019.csv', index=False)

### Atualiza a carteira online

In [73]:
#update_carteira_online(carteira.copy())

### Utilidades

- **Lucro por ação em cada mês**:

In [27]:
lucro_df = df_ativos.loc[(df_ativos['L/P'] != 0.0), ['Data', 'Código', 'L/P']].groupby(['Data', 'Código']).sum()
lucro_df

L/P
Data       Código       
2019-07-19 CVCB3  -43.00
2019-08-05 JBSS3  -16.17
2019-08-07 PETR4  -48.28
2019-09-02 ANIM3   -9.20
           B3SA3   12.00
           CVCB3   15.30
           IRBR3   77.60
           LREN3   13.00
           PCAR4  -42.75
           RAIL3   13.00
           RENT3   25.41
2019-09-09 EVEN3  -57.54
           JBSS3  -18.10
2019-10-01 BBDC3   12.00
           BBSE3   31.50
           CSNA3  -44.80
           EVEN3  -25.35
2019-11-04 MOVI3  -15.81
           SANB11  20.04
           SBSP3   57.20
2019-12-02 BBDC4  -21.15
           IRBR3   -8.68

In [77]:
print('Lucro/Prejuízo: {0}'.format( lucro_df[lucro_df['L/P'] > 0].sum()[0].round(2) ))

Lucro/Prejuízo: 277.05


- **Lucro total do ano:**

In [30]:
print('Lucro total: {0}'.format(lucro_df['L/P'].sum().round(2)))

Lucro total: -73.78


- **Lucro por mês:**

In [71]:
lucro_mes = df_ativos.loc[(df_ativos['L/P'] != 0.0), ['Data', 'L/P']]
lucro_mes['Mês'] = lucro_mes['Data'].dt.month_name()
lucro_mes = lucro_mes.groupby(['Mês'], sort=False).sum()
lucro_mes

,L/P
Mês,
July,-43.00
August,-64.45
September,28.72
October,-26.65
November,61.43
December,-29.83


### Imposto de Renda

- Os dados do imposto de renda devem ser feitos separadamente para cada corretora.
- Os dados para as ações que não foram zeradas estão listadas na carteira.
- Sempre conferir os valores com o site da B3. <- Nem sempre é o mesmo.

In [46]:
def position(df):

    cods = df['Código'].unique()

    zerada = []
    comprada = []

    for cod in cods:

        qnt = df.loc[df['Código'] == cod, 'Quantidade Atual'].tail(1).values[0]

        if qnt != 0:
            comprada.append(cod)
        else:
            zerada.append(cod)

    return zerada, comprada

In [47]:
def weighted_mean_for_duplicates(df):
    
    cod_duplicated = df['Código'].value_counts() != 1
    
    cod_unique = [not i for i in cod_duplicated]
    
    new_df = pd.DataFrame()
    
    for cod in cod_duplicated.index[cod_duplicated]:
        
        aux = df[df['Código'] == cod].reset_index(drop=True)
        
        aux.loc[0, 'Preço Atual'] = np.average(aux['Preço Atual'], weights = np.abs(aux['Quantidade de C/V']))
        aux.loc[0, 'Preço de C/V'] = np.average(aux['Preço de C/V'], weights = np.abs(aux['Quantidade de C/V']))
    
        new_df = new_df.append(aux.loc[0, :])
    
    new_df = new_df.append(df.set_index('Código').drop(cod_duplicated.index[cod_duplicated]).reset_index())
    
    new_df = new_df[['Data', 'Código', 'Preço Atual', 'Preço de C/V']].reset_index(drop=True)
    
    new_df['Preço Atual'] = new_df['Preço Atual'].round(3)
    new_df['Preço de C/V'] = new_df['Preço de C/V'].round(3)
    
    new_df['Posição'] = 'Zerada'
    
    return new_df

#### Por enquanto só serve pra ver quais ações foram zeradas

In [48]:
cols = ['Data', 'Código', 'Preço Atual', 'Preço de C/V', 'Quantidade de C/V']
zerada, comprada = position(df_ativos)

ir_df = weighted_mean_for_duplicates(df_ativos.loc[df_ativos['Código'].isin(zerada), cols]).sort_values('Código')
ir_df

,Data,Código,Preço Atual,Preço de C/V,Posição
7,2019-08-01,ANIM3,21.270,21.040,Zerada
8,2019-08-01,B3SA3,43.070,43.670,Zerada
4,2019-09-05,BBDC3,30.550,30.925,Zerada
9,2019-11-04,BBDC4,35.050,34.345,Zerada
3,2019-09-05,BBSE3,32.900,33.950,Zerada
6,2019-09-05,CSNA3,14.530,13.890,Zerada
2,2019-07-11,CVCB3,52.053,51.324,Zerada
1,2019-09-05,EVEN3,11.013,10.632,Zerada
0,2019-08-05,IRBR3,51.932,53.745,Zerada
12,2019-08-01,LREN3,49.320,49.970,Zerada
